# ANOVA  - Lab

## Introduction

In this lab, you'll get some brief practice generating an ANOVA table (AOV) and interpreting its output. You'll also perform some investigations to compare the method to the t-tests you previously employed to conduct hypothesis testing.

## Objectives

In this lab you will: 

- Use ANOVA for testing multiple pairwise comparisons 
- Interpret results of an ANOVA and compare them to a t-test

## Load the data

Start by loading in the data stored in the file `'ToothGrowth.csv'`: 

In [1]:
# Your code here
import pandas as pd

# Load the data
df = pd.read_csv('ToothGrowth.csv')
df.head()

,len,supp,dose
0,4.2,VC,0.5
1,11.5,VC,0.5
2,7.3,VC,0.5
3,5.8,VC,0.5
4,6.4,VC,0.5


## Generate the ANOVA table

Now generate an ANOVA table in order to analyze the influence of the medication and dosage:  

In [2]:
# Your code here
import statsmodels.api as sm
from statsmodels.formula.api import ols

# Generate the ANOVA table
model = ols('len ~ C(supp) * C(dose)', data=df).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
print(anova_table)


                      sum_sq    df          F        PR(>F)
C(supp)           205.350000   1.0  15.571979  2.311828e-04
C(dose)          2426.434333   2.0  91.999965  4.046291e-18
C(supp):C(dose)   108.319000   2.0   4.106991  2.186027e-02
Residual          712.106000  54.0        NaN           NaN


## Interpret the output

Make a brief comment regarding the statistics and the effect of supplement and dosage on tooth length: 

# Your comment here
The ANOVA table shows the F-statistic and p-values for the influence of supplement type and dosage on tooth length. If the p-value is less than 0.05, it indicates a significant effect.

## Compare to t-tests

Now that you've had a chance to generate an ANOVA table, its interesting to compare the results to those from the t-tests you were working with earlier. With that, start by breaking the data into two samples: those given the OJ supplement, and those given the VC supplement. Afterward, you'll conduct a t-test to compare the tooth length of these two different samples: 

In [4]:
# Your code here
# Split the data into two samples based on the supplement type
oj = df[df['supp'] == 'OJ']['len']
vc = df[df['supp'] == 'VC']['len']

Now run a t-test between these two groups and print the associated two-sided p-value: 

In [5]:
# Calculate the 2-sided p-value for a t-test comparing the two supplement groups
from scipy import stats

# Conduct a t-test
t_stat, p_val = stats.ttest_ind(oj, vc)
print("T-test p-value:", p_val)


T-test p-value: 0.06039337122412849


## A 2-Category ANOVA F-test is equivalent to a 2-tailed t-test!

Now, recalculate an ANOVA F-test with only the supplement variable. An ANOVA F-test between two categories is the same as performing a 2-tailed t-test! So, the p-value in the table should be identical to your calculation above.

> Note: there may be a small fractional difference (>0.001) between the two values due to a rounding error between implementations. 

In [6]:
# Your code here; conduct an ANOVA F-test of the oj and vc supplement groups.
# Compare the p-value to that of the t-test above. 
# They should match (there may be a tiny fractional difference due to rounding errors in varying implementations)
# List all group combinations
groups = df.groupby(['supp', 'dose'])['len']
for group in groups:
    group_name = group[0]
    data = group[1]
    print(group_name)


('OJ', 0.5)
('OJ', 1.0)
('OJ', 2.0)
('VC', 0.5)
('VC', 1.0)
('VC', 2.0)


## Run multiple t-tests

While the 2-category ANOVA test is identical to a 2-tailed t-test, performing multiple t-tests leads to the multiple comparisons problem. To investigate this, look at the various sample groups you could create from the 2 features: 

In [7]:
for group in df.groupby(['supp', 'dose'])['len']:
    group_name = group[0]
    data = group[1]
    print(group_name)

('OJ', 0.5)
('OJ', 1.0)
('OJ', 2.0)
('VC', 0.5)
('VC', 1.0)
('VC', 2.0)


While bad practice, examine the effects of calculating multiple t-tests with the various combinations of these. To do this, generate all combinations of the above groups. For each pairwise combination, calculate the p-value of a 2-sided t-test. Print the group combinations and their associated p-value for the two-sided t-test.

In [8]:
# Your code here; reuse your t-test code above to calculate the p-value for a 2-sided t-test
# for all combinations of the supplement-dose groups listed above. 
# (Since there isn't a control group, compare each group to every other group.)
from itertools import combinations

# Generate all combinations of the groups
group_data = [group[1] for group in groups]
group_names = [group[0] for group in groups]
combinations = list(combinations(range(len(group_data)), 2))

# Calculate p-values for each pairwise combination
for comb in combinations:
    group1 = group_data[comb[0]]
    group2 = group_data[comb[1]]
    t_stat, p_val = stats.ttest_ind(group1, group2)
    print(f"Comparison: {group_names[comb[0]]} vs {group_names[comb[1]]}, p-value: {p_val}")


Comparison: ('OJ', 0.5) vs ('OJ', 1.0), p-value: 8.357559281443774e-05
Comparison: ('OJ', 0.5) vs ('OJ', 2.0), p-value: 3.4018585295016156e-07
Comparison: ('OJ', 0.5) vs ('VC', 0.5), p-value: 0.005303661339923052
Comparison: ('OJ', 0.5) vs ('VC', 1.0), p-value: 0.04223992429368205
Comparison: ('OJ', 0.5) vs ('VC', 2.0), p-value: 7.025409196997986e-06
Comparison: ('OJ', 1.0) vs ('OJ', 2.0), p-value: 0.03736279585664383
Comparison: ('OJ', 1.0) vs ('VC', 0.5), p-value: 1.3372624230559434e-08
Comparison: ('OJ', 1.0) vs ('VC', 1.0), p-value: 0.0007807261651774468
Comparison: ('OJ', 1.0) vs ('VC', 2.0), p-value: 0.09583711277517494
Comparison: ('OJ', 2.0) vs ('VC', 0.5), p-value: 1.3381068810881244e-11
Comparison: ('OJ', 2.0) vs ('VC', 1.0), p-value: 2.3131084633597503e-07
Comparison: ('OJ', 2.0) vs ('VC', 2.0), p-value: 0.9637097790041267
Comparison: ('VC', 0.5) vs ('VC', 1.0), p-value: 6.492264598157612e-07
Comparison: ('VC', 0.5) vs ('VC', 2.0), p-value: 4.95728565843888e-09
Comparison: (

## Summary

In this lesson, you implemented the ANOVA technique to generalize testing methods to multiple groups and factors.